# Parallelism in Python

### John Kirkham

# The Problem

* Typical threading models are hard for (new) users to understand
* Easy to run into difficult to debug scenarios (e.g. deadlocking, race conditions, etc.)
* Implementation often becomes tied to a certain scale (e.g. multithreaded code -> cluster parallelized code)
* How could this be done better?

# Task-based parallelism

* Describe the pieces of the computation
* Relate these pieces to each other
* Use a scheduler to perform the computation

# Common implementations

* Dask
* ipyparallel
* Luigi

# Dask - Introducing a Task Graph

![]( images/pipeline.svg )

# Dask - A short example

![]( images/dask_example1.svg )

# Dask - A short example


```python
In [1]: import dask

In [2]: a = [0, 1, 2, 3, 4]

In [3]: d = {"a": a, "b": (sum, "a")}

In [4]: dask.get(d, "a")
Out[4]: [0, 1, 2, 3, 4]

In [5]: dask.get(d, "b")
Out[5]: 10
```